In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain_ollama import OllamaLLM
import pandas as pd
from langchain_community.llms import Ollama
import numpy as np
import pandas as pd

In [ ]:
#Importing OpenAI's token counter
import tiktoken
encoding = tiktoken.get_encoding("o200k_base")
encoding = tiktoken.encoding_for_model("gpt-4o")

In [ ]:
#Function for token counter
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [ ]:
#stitching 
df = pd.read_csv(r'C:\Users\yens01\Projects\Frailty\Frailty Notes 60D 5.27.2025.csv')

In [ ]:
#Importing OpenAI's token counter
import tiktoken
encoding = tiktoken.get_encoding("o200k_base")
encoding = tiktoken.encoding_for_model("gpt-4o")
#Function for token counter
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens
#stitching the notes by person_id, MRN and merging with cohortdf for valiated RAI score
stitched = df.groupby(['person_id','MRN','Validated_RAI_Score','Dt','Age'])['note_text'].agg('/n/n'.join).reset_index()
#Applying counter to df
stitched['token_count'] = stitched['note_text'].apply(lambda x: num_tokens_from_string(x, "o200k_base"))

In [ ]:
#Exclude high token count
high_count_stitched = stitched[stitched['token_count'] > 100000].reset_index(drop = True)
stitched = stitched[stitched['token_count'] <= 100000].reset_index(drop = True)

In [ ]:
#Connecting to Minervas ollama
LLM = OllamaLLM(model="llama3.3", temperature=0, base_url= MINERVA_API, num_ctx = 100000)


In [ ]:
# Prompt 
regular_prompt = PromptTemplate(
    template="""
You are a medical doctor and expert in analyzing clinical notes.

Patient age: {age_input}
Return a valid JSON key "age"

Sex:
Determine if the patient is male or female.
Return a valid JSON key "sex" with value "male" or "female".

Sex Category Score:
Based on the patient's sex determination, assign a sex category score as follows:
- If the patient is male, score = 5
- If the patient is female, score = 0
Return a valid JSON key "sex_category_score". 

Cancer:
Based on the clinical notes below, make the following determinations:
1. Whether the patient has active cancer and specify the type of cancer.
2. Whether the patient has any type of active skin cancer.
3. Whether the patient has any type of active melanoma.
Return the following valid JSON keys::
- "active_cancer" (boolean)
- "cancer_type" (string, or empty string if none)
- "active_skin_cancer" (boolean)
- "active_melanoma_cancer" (boolean)

Age and Cancer Score:
Calculate "age_and_cancer_category_score" based on age and active cancer status using the following rules:

- If age is between 0 and 69:
  - If active cancer is true → score = 20
  - If active cancer is false → score = 2

- If age is between 70 and 74:
  - If active cancer is true → score = 19
  - If active cancer is false → score = 3

- If age is between 75 and 79:
  - If active cancer is true → score = 18
  - If active cancer is false → score = 4

- If age is between 80 and 84:
  - If active cancer is true → score = 17
  - If active cancer is false → score = 5

- If age is between 85 and 89:
  - If active cancer is true → score = 16
  - If active cancer is false → score = 6

- If age is between 90 and 94:
  - If active cancer is true → score = 15
  - If active cancer is false → score = 7

- If age is between 95 and 99:
  - If active cancer is true → score = 14
  - If active cancer is false → score = 8

- If age is 100 or older:
  - If active cancer is true → score = 13
  - If active cancer is false → score = 9
Return a valid JSON key "age_and_cancer_category_score"

Clinical Conditions:
Based on the clinical notes, determine the following. For each, return true if there is evidence, otherwise false:
- unintentional weight loss of more than 10 lbs in the past 3 months
- renal failure
- chronic or congestive heart failure
- poor appetite
- shortness of breath at rest
Return ONLY the following valid JSON keys:
- "unintentional_weight_loss" (boolean)
- "renal_failure" (boolean)
- "heart_failure" (boolean)
- "shortness_of_breath" (boolean)

Medical Comorbidities Score:
Based on the clinical conditions determined above, calculate "co_morbidities_category_score" as follows:
- Add 5 points if unintentional_weight_loss is true
- Add 6 points if renal_failure is true
- Add 4 points if heart_failure is true
- Add 4 points if poor_appetite is true
- Add 8 points if shortness_of_breath is true
Return a valid JSON key "co_morbidities_category_score"

Residence Status:
Read the clinical notes and determine if there is evidence that the patient resides in:
- a skilled nursing facility
- assisted living
- a nursing home
Return the following valid JSON keys:
- "skilled_nursing_facility"
- "assisted_living"
- "nursing_home"
Set each value to true if there is evidence in the clinical notes, otherwise false.

Cognition, Residence, and Activity of Daily Living Category Score:
Calculate "cognition_residence_activity_daily_living_category_score" based on the following rule:
- If any of the following are true: skilled_nursing_facility, assisted_living, or nursing_home → score = 8
- If all are false → score = 0
Return a valid JSON key "cognition_residence_activity_daily_living_category_score"

Cognition:
Based on the clinical notes below, determine if the patient's cognitive skills or status has deteriorated over the past 3 months.
Return a valid JSON "cognitive_decline" (value must be either true or false)

Level of Functioning:
Based on the clinical notes below, determine the patient's current level of functioning in the following areas:
- Mobility/locomotion
- Eating
- Toilet use
- Hygiene
For each, return one of the following values only:
"independent", "supervised", "limited assistance", "extensive assistance", or "total dependence".
Return the following valid JSON keys:
- "mobility_locomotion"
- "eating"
- "toilet_use"
- "hygiene"

Activities of Daily Living & Cognitive Decline Category Score:
Assign numeric scores to each functional category based on the following mapping:
- "independent" → 0  
- "supervised" → 1  
- "limited assistance" → 2  
- "extensive assistance" → 3  
- "total dependence" → 4  
Calculate the sum of the scores for mobility_locomotion, eating, toilet_use, and hygiene.

Based on cognitive_decline and the summed score:
- If cognitive_decline is true and sum = 0, subtract 2 points.
- If cognitive_decline is true and sum = 1 or 2, subtract 1 point.
- If cognitive_decline is true and sum = 3 or 4, add 0 points.
- If cognitive_decline is true and sum between 5 and 7, add 1 point.
- If cognitive_decline is true and sum = 8 or 9, add 2 points.
- If cognitive_decline is true and sum = 10 or 11, add 3 points.
- If cognitive_decline is true and sum = 12 or 13, add 4 points.
- If cognitive_decline is true and sum between 14 and 16, add 5 points.
- If cognitive_decline is false, add 0 points.

Return the valid JSON key "activity_daily_living_cognitive_decline_category_score"

RAI score:
Calculate the RAI_score step by step:

Step 1: Add age_and_cancer_category_score and sex_category_score = X  
Step 2: Add co_morbidities_category_score to X = Y  
Step 3: Add cognition_residence_activity_daily_living_category_score to Y = Z  
Step 4: Add activity_daily_living_cognitive_decline_category_score to Z = RAI_score  

Return the valid JSON key "RAI_score.

Return ONLY a valid JSON object with all the keys below:
- "sex" (string)
- "sex_category_score" (integer)
- "active_cancer" (boolean)
- "cancer_type" (string)
- "active_skin_cancer" (boolean)
- "active_melanoma_cancer" (boolean)
- "age_and_cancer_category_score" (integer)
- "unintentional_weight_loss" (boolean)
- "renal_failure" (boolean)
- "heart_failure" (boolean)
- "poor_appetite" (boolean)
- "shortness_of_breath" (boolean)
- "skilled_nursing_facility" (boolean)
- "assisted_living" (boolean)
- "nursing_home" (boolean)
- "cognitive_decline" (boolean)
- "mobility_locomotion" (string)
- "eating" (string)
- "toilet_use" (string)
- "hygiene" (string)
- "cognition_residence_activity_daily_living_category_score" (integer)
- "activity_daily_living_cognitive_decline_category_score" (integer)
- "co_morbidities_category_score" (integer)
- "RAI_score" (integer)

Do NOT include any explanation, markdown formatting, or extra text beyond the JSON object.

CLINICAL NOTES:
{clinical_notes}
""",
    input_variables=["text_doc","age_input"]
)

In [ ]:
stitched_llm = stitched.copy()

In [ ]:
regular_chain = regular_prompt | LLM | JsonOutputParser()

In [ ]:
def wholeprompt(notes, age_input):
    regular_chain = regular_prompt | LLM | JsonOutputParser()
    input_data = {
        "clinical_notes": notes,
        "age_input": age_input
    }
    try:
        output = regular_chain.invoke(input_data)
    except Exception as e:
        return "Invalid Output"
    else:
        return output 

In [ ]:
def func(row):
    print(f"Calculating for MRN : {row['MRN']}")
    return wholeprompt(row['note_text'], row['Age'])

In [ ]:
#Running the LLM 
stitched_llm['llm_output'] = stitched_llm.apply(func, axis=1)

In [ ]:
stitched_llm_test = stitched_llm.copy()
stitched_llm_test = stitched_llm_test.reset_index(drop = True)

In [ ]:
mrn = stitched_llm_test[stitched_llm_test['llm_output'] == 'Invalid Output']['MRN']
stitched_llm_test[stitched_llm_test['llm_output'] == 'Invalid Output']

In [ ]:
stitched_llm_test = stitched_llm_test[stitched_llm_test['llm_output'] != 'Invalid Output']

In [ ]:
stitched_llm_test

In [ ]:
# Expand 'llm_output' dictionary into separate columns
llm_expanded = stitched_llm_test['llm_output'].apply(pd.Series)

# Merge back into original DataFrame
stitched_llm_test = stitched_llm_test.join(llm_expanded)

# Get the new column names
llm_output_columns = llm_expanded.columns.tolist()
# Adding 'llm' to column names
llm_output_columns_add = ['llm_' + col for col in llm_output_columns] 
rename_dict = dict(zip(llm_output_columns, llm_output_columns_add))
stitched_llm_test = stitched_llm_test.rename(columns=rename_dict)


In [ ]:
stitched_llm_test.loc[:, 'llm_age_sex_cancer_score'] = (
    stitched_llm_test['llm_sex_category_score'] + stitched_llm_test['llm_age_and_cancer_category_score']
)


In [ ]:
stitched_llm_test['llm_RAI_score'] = stitched_llm_test.apply(
    lambda row: row['llm_rai_score'] if pd.isna(row['llm_RAI_score']) and pd.notna(row['llm_rai_score']) else row['llm_RAI_score'],
    axis=1
)

In [ ]:
# Determine frail/not frail base off LLM output. Frail if RAI >= 21 
stitched_llm_test['LLM_frail_yn'] = stitched_llm_test['llm_RAI_score'].apply(lambda x: 1 if x >= 21 else 0)

In [ ]:
#replace some patients are missing morbidities but have the parts, stitching together
for i in range(len(stitched_llm_test)):
    mrn = stitched_llm_test.iloc[i]['MRN']
    if pd.notna(stitched_llm_test.iloc[i]['llm_co_morbidities_category_score']):
        stitched_llm_test.loc[stitched_llm_test['MRN'] == mrn, 'llm_co_morbidities_category_score'] = stitched_llm_test.iloc[i]['llm_co_morbidities_category_score']
    if pd.isna(stitched_llm_test.iloc[i]['llm_co_morbidities_category_score']):
        score = 0.0
        if stitched_llm_test.iloc[i]['llm_unintentional_weight_loss'] == True:
            score += 5.0
        if stitched_llm_test.iloc[i]['llm_renal_failure'] == True:
            score += 6.0
        if stitched_llm_test.iloc[i]['llm_heart_failure'] == True:
            score += 4.0
        if stitched_llm_test.iloc[i]['llm_poor_appetite'] == True:
            score += 4.0
        if stitched_llm_test.iloc[i]['llm_shortness_of_breath'] == True:
            score += 8.0
        stitched_llm_test.loc[stitched_llm_test['MRN'] == mrn, 'llm_co_morbidities_category_score'] = score

In [ ]:
to_csv = stitched_json_error.drop(columns=['note_text','llm_output'])

In [ ]:
# to_csv.to_csv(r'C:\Users\yens01\Projects\Frailty\LLM_OUTPUT\stitched_final_llm_single_llama3.3 FINAL 6.1.2025.csv', index= False)

In [ ]:
# read csv
stitched_llm_test = pd.read_csv(r'C:\Users\yens01\Projects\Frailty\LLM_OUTPUT\FINAL OUTPUT\stitched_final_llm_single_llama3.3 FINAL 6.1.2025.csv')

In [ ]:
#Bringing back the patient reported RAI scores 
redcap_cohort = pd.read_excel(r'C:\Users\yens01\Projects\Frailty\REDCap_Frailty_Cohort_Final 4.4.2025.xlsx')

In [ ]:
# Matching same dtype
stitched_llm_test['MRN'] = stitched_llm_test['MRN'].astype(str).str.strip()
redcap_cohort['MRN'] = redcap_cohort['MRN'].astype(str).str.strip()

# Wanted columns
redcap_cols = [
    'MRN', 'Total RAI Score', 'Age', "Patient's age score with cancer included", 'Sex',
    'Unintentional weight loss in the past three months?', 'Renal failure?',
    'Chronic/congestive heart failure?', 'Poor appetite?', 'Shortness of breath (at rest)?',
    'Reside in a setting other than independent living?', 'Mobility/Locomotion',
    'Eating', 'Toilet Use', 'Personal Hygeine',
    'Have cognitive skills/status deteriorated over the past 3 months?',
    'ADL Score (sum of ADL scores) with/without cognitive decline consideration'
]

# Selecting the specified columns
redcap_cohort[redcap_cols]

In [ ]:
merged = pd.merge(stitched_llm_test, redcap_cohort[redcap_cols], on= 'MRN', how = 'inner')
merged

In [ ]:
merged['Validated_frail_yn'] = merged['Validated_RAI_Score'].apply(lambda x: 1 if x >= 21 else 0)

In [ ]:
for col in merged.columns:
    if col.startswith('llm'):
        error_count = (merged[col] == 'JSON ERROR').sum()
        if error_count > 0:
            print(f"{col} : {error_count}")

In [ ]:
merged['Validated_frail_yn'].value_counts()

In [ ]:
#mean of not frail patients 
merged[['Validated_RAI_Score', 'llm_RAI_score']][merged['Validated_frail_yn'] == 0].mean()

In [ ]:
#mean of frail patients 
merged[['Validated_RAI_Score', 'llm_RAI_score']][merged['Validated_frail_yn'] == 1].mean()

In [ ]:
print("missing llm_RAI_score:",len(merged[merged[['llm_RAI_score']].isna().any(axis=1)][['llm_RAI_score','MRN']].index.tolist()))
print("missing llm_co_morbidities_category_score:",len(merged[merged[['llm_co_morbidities_category_score']].isna().any(axis=1)][['llm_RAI_score','MRN']].index.tolist()))
print("missing llm_age_sex_cancer_score:",len(merged[merged[['llm_age_sex_cancer_score']].isna().any(axis=1)][['llm_RAI_score','MRN']].index.tolist()))
print("missing llm_activity_daily_living_cognitive_decline_category_score:",len(merged[merged[['llm_activity_daily_living_cognitive_decline_category_score']].isna().any(axis=1)][['llm_RAI_score','MRN']].index.tolist()))


In [ ]:
stitched_llm_test[stitched_llm_test['MRN'] =='7520588']

In [ ]:
llm_cols = [col for col in merged.columns if col.startswith('llm')]
error_mask = (merged[llm_cols] == 'JSON ERROR')
error_indices = merged[error_mask.any(axis=1)].index.tolist()
print("Unique rows:",len(error_indices))

In [ ]:
merged_rsme = merged.dropna(subset=['llm_RAI_score','llm_co_morbidities_category_score','llm_age_sex_cancer_score']).reset_index(drop = True)

In [ ]:
from sklearn.metrics import classification_report
target_names = ['Not Frail', 'Frail']
y_true = merged_rsme['Validated_frail_yn']
y_pred = merged_rsme['LLM_frail_yn']
print("qwen2.5 7b Single Prompt LLM Frail Classification Report from RAI Survey:")
print(classification_report(y_true,y_pred, target_names = target_names))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred, labels=[0, 1])  # Ensure correct class order
cm = confusion_matrix(y_true, y_pred, labels=[1, 0])
classes = ['Frail', 'Not Frail']

fig, ax = plt.subplots()
im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
ax.figure.colorbar(im, ax=ax)

ax.set(xticks=np.arange(len(classes)),
       yticks=np.arange(len(classes)),
       xticklabels=classes,
       yticklabels=classes,
       xlabel='LLM Classification',
       ylabel='Validated Classification')

# Match TP/FP/FN/TN to new order
labels = np.array([['TP', 'FN'], ['FP', 'TN']])
thresh = cm.max() / 2.

for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, f"{labels[i, j]}\n{cm[i, j]}",
                ha="center", va="center",
                color="white" if cm[i, j] > thresh else "black")

plt.title("Confusion Matrix")
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import average_precision_score
pr_auc = average_precision_score(y_true, y_pred)
print(f"PR AUC: {pr_auc}")

In [ ]:
#pull class report metrics for metricsdf
accuracy = classification_report(y_true,y_pred, target_names = target_names, output_dict= True).get('accuracy')
accuracy

In [ ]:
precision = classification_report(y_true,y_pred, target_names = target_names, output_dict= True).get('Frail').get('precision')
recall = classification_report(y_true,y_pred, target_names = target_names, output_dict= True).get('Frail').get('recall')
f1 = classification_report(y_true,y_pred, target_names = target_names, output_dict= True).get('Frail').get('f1-score')


In [ ]:
classification_report(y_true,y_pred, target_names = target_names, output_dict= True)

In [ ]:
from sklearn.metrics import root_mean_squared_error
y_true = merged_rsme['Validated_RAI_Score']
y_pred = merged_rsme['llm_RAI_score']
print("Single Prompt LLM:")
print("RAI score RMSE:",root_mean_squared_error(y_true,y_pred))
rai_score_rmse = root_mean_squared_error(y_true,y_pred)

In [ ]:
redcap_cols = [
    'MRN', 'Total RAI Score', 'Age', "Patient's age score with cancer included", 'Sex',
    'Unintentional weight loss in the past three months?', 'Renal failure?',
    'Chronic/congestive heart failure?', 'Poor appetite?', 'Shortness of breath (at rest)?',
    'Reside in a setting other than independent living?', 'Mobility/Locomotion',
    'Eating', 'Toilet Use', 'Personal Hygeine',
    'Have cognitive skills/status deteriorated over the past 3 months?',
    'ADL Score (sum of ADL scores) with/without cognitive decline consideration'
]

for i in range(len(merged_rsme)):
    sex_score = 0
    age_score = 0
    if merged_rsme.iloc[i]['Sex'] == 'Male':
        sex_score = 5
    if pd.isna(merged_rsme.iloc[i]["Patient's age score with cancer included"]):
        pass
    else:
        age_score = merged_rsme.iloc[i]["Patient's age score with cancer included"]

    merged_rsme.at[i,'Validated_Age_Sex_Cancer_Score'] = (age_score + sex_score)

In [ ]:
#merged_rsme[['Validated_Age_Sex_Cancer_Score','llm_age_sex_cancer_score']]
y_true = merged_rsme['Validated_Age_Sex_Cancer_Score']
y_pred = merged_rsme['llm_age_sex_cancer_score']

print(merged_rsme[['Validated_Age_Sex_Cancer_Score','llm_age_sex_cancer_score']])
print("Age Sex Cancer Score RMSE:",root_mean_squared_error(y_true,y_pred))
age_sex_cancer_score_rmse = root_mean_squared_error(y_true,y_pred)

In [ ]:
#consolidate llm ADL score
for i in range(len(merged_rsme)):
    if pd.isna(merged_rsme.at[i, 'llm_activity_daily_living_cognitive_decline_category_score']):
        if not pd.isna(merged_rsme.at[i, 'llm_activities_of_daily_living_cognitive_decline_category_score']):
            merged_rsme.at[i, 'llm_ADL_score'] = merged_rsme.at[i, 'llm_activities_of_daily_living_cognitive_decline_category_score']
    else:
        merged_rsme.at[i, 'llm_ADL_score'] = merged_rsme.at[i, 'llm_activity_daily_living_cognitive_decline_category_score']

In [ ]:
merged_rsme[merged_rsme['llm_ADL_score'].isna()].any(axis=1)

In [ ]:
merged_rsme['llm_ADL_score']
merged_rsme[["ADL Score (sum of ADL scores) with/without cognitive decline consideration"]]
y_true = merged_rsme["ADL Score (sum of ADL scores) with/without cognitive decline consideration"]
y_pred = merged_rsme['llm_ADL_score']

#print(merged[["ADL Score (sum of ADL scores) with/without cognitive decline consideration",'llm_adlcd_score']])
print("ADL score RMSE:",root_mean_squared_error(y_true,y_pred))
adl_score_rmse = root_mean_squared_error(y_true,y_pred)

In [ ]:
#adding the medical comorbidities together 

import re

def extract_number(text):
    numbers = re.findall(r'[0-9]+', str(text))  # Find all numbers
    return int(''.join(numbers)) # if numbers else 0  # Convert to int or return 0

# Apply function to each relevant column and sum the values
merged_rsme['Validated_medical_comorbidities_score'] = (
    merged_rsme['Unintentional weight loss in the past three months?'].apply(extract_number) +
    merged_rsme['Renal failure?'].apply(extract_number) +
    merged_rsme['Chronic/congestive heart failure?'].apply(extract_number) +
    merged_rsme['Poor appetite?'].apply(extract_number) +
    merged_rsme['Shortness of breath (at rest)?'].apply(extract_number)
)


In [ ]:
#Checked, no 
merged_rsme[
    merged_rsme[['llm_co_morbidities_category_score']].isna().any(axis=1)
].index.tolist()


In [ ]:
y_true = merged_rsme["Validated_medical_comorbidities_score"]
y_pred = merged_rsme['llm_co_morbidities_category_score']

#print(merged[["ADL Score (sum of ADL scores) with/without cognitive decline consideration",'llm_adlcd_score']])
print("medical_comorbidities_score RMSE:",root_mean_squared_error(y_true,y_pred))
medical_comorbidities_score_rmse = root_mean_squared_error(y_true,y_pred)

In [ ]:
#translating cradl 

import re

def extract_number(text):
    numbers = re.findall(r'[0-9]+', str(text))  # Find all numbers
    return int(''.join(numbers)) # if numbers else 0  # Convert to int or return 0

# Apply function to each relevant column and sum the values
merged_rsme['Validated_CRADL_score'] = merged_rsme['Reside in a setting other than independent living?'].apply(extract_number)

In [ ]:
merged_rsme[merged_rsme['llm_cognition_residence_activity_daily_living_category_score'].isna()]

In [ ]:
# #consolidate llm_CRADL_score
merged_rsme['llm_CRADL_score'] = merged_rsme.apply(
    lambda row: row['llm_cognition_residence_activity_of_daily_living_category_score'] if pd.isna(row['llm_cognition_residence_activity_daily_living_category_score']) and pd.notna(row['llm_cognition_residence_activity_of_daily_living_category_score']) else row['llm_cognition_residence_activity_daily_living_category_score'],
    axis=1
)

In [ ]:
merged_rsme[
    merged_rsme[['llm_CRADL_score']].isna().any(axis=1)][['llm_CRADL_score','MRN']]

In [ ]:
y_true = merged_rsme["Validated_CRADL_score"]
y_pred = merged_rsme['llm_CRADL_score']

#print(merged[["ADL Score (sum of ADL scores) with/without cognitive decline consideration",'llm_adlcd_score']])
print("CRADL RMSE:",root_mean_squared_error(y_true,y_pred))
cradl_score_rmse = root_mean_squared_error(y_true,y_pred)

In [ ]:
#storing agentic metrics in a DF:
metrics_df = pd.DataFrame( columns = ['LLM Model','LLM System','RAI score RMSE','Age Sex Cancer Score RMSE', 'ADL Score RMSE', 'Medical Co-Morbidities Score RMSE'])
metrics_df = pd.DataFrame([{'LLM Model' :'llama3.3'
                            ,'LLM System' : 'Single'
                            ,'Accuracy' : accuracy
                            ,'Precision' : precision
                            ,'Recall' : recall
                            ,'F1-score': f1
                            ,'PR AUC': pr_auc
                            ,'Spearman’s ρ': corr
                            ,'RAI score RMSE' : rai_score_rmse
                            ,'Age Sex Cancer Score RMSE': age_sex_cancer_score_rmse
                            ,'ADL Score RMSE' : adl_score_rmse
                            , 'Medical Co-Morbidities Score RMSE' : medical_comorbidities_score_rmse
                            ,'CRADL RMSE': cradl_score_rmse}])
metrics_df

In [ ]:
sub_score_df = merged_rsme[['MRN','Validated_RAI_Score','llm_RAI_score','Validated_Age_Sex_Cancer_Score','llm_age_sex_cancer_score','Validated_medical_comorbidities_score','llm_co_morbidities_category_score','Validated_CRADL_score','llm_CRADL_score','ADL Score (sum of ADL scores) with/without cognitive decline consideration','llm_ADL_score']]
# sub_score_df.to_csv(r'C:\Users\yens01\Projects\Frailty\RESULTS\Sub RAI Score results\llama3.1 single sub score.csv')

In [ ]:

sub_score_df.to_csv(r'C:\Users\yens01\Projects\Frailty\RESULTS\Sub RAI Score results\llama3.3 single sub score.csv')

In [ ]:
# metrics_df.to_csv(r'C:\Users\yens01\Projects\Frailty\METRICS\qwen2.5 7b single approach results FINAL 6.1.2025.csv')

In [ ]:
from scipy.stats import spearmanr

# Example: correlation between LLM score and true RAI score
corr, p_value = spearmanr(merged_rsme['llm_RAI_score'], merged_rsme['Validated_RAI_Score'])

print(f"llama3.3 Single Prompt Approach Spearman correlation: {corr:.3f}, p-value: {p_value:.3g}")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.scatterplot(x='llm_RAI_score', y='Validated_RAI_Score', data=merged, legend= "full")
plt.legend(title=f'Spearman’s ρ = {corr:.2f}', loc = 'lower right')
plt.title(f"llama3.3 Single LLM System")
plt.xlabel("LLM RAI Score")
plt.ylabel("Patient-Reported RAI Score")
plt.grid(True)
plt.show()

####END####